In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
SPs = [f'D{day}{rep}' for day in ['0','4','8'] for rep in ['a','b','c']]

In [48]:
for SP in SPs:
    codon_count = pd.read_csv(f'/Data_2/Daehwa/Adipocyte/Analysis/Ribosome_stalling/v20230730/{SP}.codon-count.tsv', sep='\t', usecols=['transcript_id','reltostart-asite','count'])
    codon_count = codon_count.rename(columns={'count':f'{SP}_n'})

    if SP=='D0a':
        total_codon_count = codon_count.copy()
    else:
        total_codon_count = pd.merge(total_codon_count, codon_count, on=['transcript_id','reltostart-asite'], how='outer')

total_codon_count = total_codon_count.sort_values(['transcript_id','reltostart-asite']).reset_index(drop=True).replace(np.nan, 0)
display(total_codon_count)

,reltostart-asite,transcript_id,D0a_n,D0b_n,D0c_n,D4a_n,D4b_n,D4c_n,D8a_n,D8b_n,D8c_n
0,33,ENSMUST00000000001.5,5.0,3.0,0.0,1.0,2.0,0.0,1.0,0.0,4.0
1,36,ENSMUST00000000001.5,1.0,0.0,1.0,0.0,0.0,4.0,0.0,3.0,1.0
2,39,ENSMUST00000000001.5,6.0,0.0,0.0,1.0,0.0,0.0,5.0,4.0,1.0
3,42,ENSMUST00000000001.5,0.0,0.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0
4,45,ENSMUST00000000001.5,5.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
3649604,1512,ENSMUST00000239563.2,6.0,1.0,0.0,1.0,1.0,5.0,0.0,0.0,2.0
3649605,1518,ENSMUST00000239563.2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3649606,1521,ENSMUST00000239563.2,0.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,0.0
3649607,1524,ENSMUST00000239563.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [94]:
slct_codon_table = pd.DataFrame(columns=['transcript_id','reltostart-asite','codon_num']+[SP+'_n' for SP in SPs]+[SP+'_N' for SP in SPs])
for T_ID, table in total_codon_count.groupby('transcript_id'):
    continue_flag = False

    table = table.set_index('reltostart-asite')
    table = table.reindex(range(table.index.min(), table.index.max()+1, 3), fill_value=0).reset_index()
    table['transcript_id'] = T_ID
    # print(T_ID)

    for day in ['D0','D4','D8']:
        avg_sum = table[[f'{day}{rep}_n' for rep in ['a','b','c']]].sum().sum()/3
        # print(f'{day} {avg_sum:.1f}, {avg_sum / len(table):.3f}')
        if avg_sum / len(table) < 0.5 : continue_flag = True
        if avg_sum < 100 : continue_flag = True
    if continue_flag : continue

    for SP in SPs:
        table[f'{SP}_N'] = sum(table[f'{SP}_n'])
    table['codon_num'] = len(table)
    
    slct_codon_table = pd.concat([slct_codon_table, table])

slct_codon_table.to_csv('adi_codon-count.tsv', sep='\t', index=False)
display(slct_codon_table)

,transcript_id,reltostart-asite,codon_num,D0a_n,D0b_n,D0c_n,D4a_n,D4b_n,D4c_n,D8a_n,...,D8c_n,D0a_N,D0b_N,D0c_N,D4a_N,D4b_N,D4c_N,D8a_N,D8b_N,D8c_N
0,ENSMUST00000000001.5,33,334,5.0,3.0,0.0,1.0,2.0,0.0,1.0,...,4.0,1424.0,1259.0,1347.0,1243.0,1620.0,1487.0,1015.0,1152.0,1300.0
1,ENSMUST00000000001.5,36,334,1.0,0.0,1.0,0.0,0.0,4.0,0.0,...,1.0,1424.0,1259.0,1347.0,1243.0,1620.0,1487.0,1015.0,1152.0,1300.0
2,ENSMUST00000000001.5,39,334,6.0,0.0,0.0,1.0,0.0,0.0,5.0,...,1.0,1424.0,1259.0,1347.0,1243.0,1620.0,1487.0,1015.0,1152.0,1300.0
3,ENSMUST00000000001.5,42,334,0.0,0.0,0.0,4.0,1.0,1.0,0.0,...,0.0,1424.0,1259.0,1347.0,1243.0,1620.0,1487.0,1015.0,1152.0,1300.0
4,ENSMUST00000000001.5,45,334,5.0,0.0,0.0,0.0,3.0,1.0,1.0,...,2.0,1424.0,1259.0,1347.0,1243.0,1620.0,1487.0,1015.0,1152.0,1300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476,ENSMUST00000239525.1,10461,3481,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,7838.0,9448.0,12828.0,3860.0,11417.0,4005.0,1922.0,1354.0,2324.0
3477,ENSMUST00000239525.1,10464,3481,0.0,1.0,2.0,3.0,1.0,0.0,0.0,...,0.0,7838.0,9448.0,12828.0,3860.0,11417.0,4005.0,1922.0,1354.0,2324.0
3478,ENSMUST00000239525.1,10467,3481,13.0,1.0,4.0,0.0,3.0,2.0,0.0,...,0.0,7838.0,9448.0,12828.0,3860.0,11417.0,4005.0,1922.0,1354.0,2324.0
3479,ENSMUST00000239525.1,10470,3481,12.0,9.0,0.0,0.0,7.0,1.0,3.0,...,1.0,7838.0,9448.0,12828.0,3860.0,11417.0,4005.0,1922.0,1354.0,2324.0


# Stalling score

In [113]:
# slct_codon_table = pd.read_csv('/Data_2/Daehwa/Adipocyte/Analysis/Ribosome_stalling/v20230730/adi_codon-count.tsv', sep='\t')

stall_score = slct_codon_table[['transcript_id','reltostart-asite']].copy()
for SP in [f'D{day}{rep}' for day in ['0','4','8'] for rep in ['a','b','c']]:
    stall_score[SP] = slct_codon_table[SP+'_n'] / (slct_codon_table[SP+'_N'].replace(0, np.nan) / slct_codon_table['codon_num'])
stall_score = stall_score.replace(np.nan, 0)

codon_table = pd.DataFrame(columns=['transcript_id','asite','reltostart-asite','codon-asite','aa-asite'])
for SP in [day+rep for day in ['0','4','8'] for rep in ['a','b','c']]:
    codon_data = pd.read_csv(f'/Data_2/Jun/Adipocytes/rpf/novaseq/D{SP}.rep.codons.data.txt', sep='\t')
    codon_data = codon_data[['transcript_id','asite','reltostart-asite','codon-asite','aa-asite']]
    codon_table = pd.merge(codon_table, codon_data, on=['transcript_id','asite','reltostart-asite','codon-asite','aa-asite'], how='outer')

stall_score = pd.merge(codon_table, stall_score, on=['transcript_id','reltostart-asite'])

with open("adi_stall-score.df.pickle","wb") as fw:
    pickle.dump(stall_score, fw)
stall_score.to_csv('adi_stall-score.tsv', sep='\t', index=False)
display(stall_score)

,transcript_id,asite,reltostart-asite,codon-asite,aa-asite,D0a,D0b,D0c,D4a,D4b,D4c,D8a,D8b,D8c
0,ENSMUST00000000001.5,174,33,GCG,A,1.172753,0.79587,0.000000,0.268705,0.412346,0.000000,0.329064,0.000000,1.027692
1,ENSMUST00000000001.5,177,36,GUG,V,0.234551,0.00000,0.247958,0.000000,0.000000,0.898453,0.000000,0.869792,0.256923
2,ENSMUST00000000001.5,180,39,GAG,E,1.407303,0.00000,0.000000,0.268705,0.000000,0.000000,1.645320,1.159722,0.256923
3,ENSMUST00000000001.5,186,45,AGC,S,1.172753,0.00000,0.000000,0.000000,0.618519,0.224613,0.329064,0.000000,0.513846
4,ENSMUST00000000001.5,189,48,AAG,K,0.234551,0.53058,0.247958,0.000000,0.000000,0.000000,0.987192,0.289931,0.256923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885452,ENSMUST00000239525.1,4271,4140,UCU,S,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.497849
1885453,ENSMUST00000239525.1,5324,5193,GCG,A,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.497849
1885454,ENSMUST00000239525.1,8915,8784,CGC,R,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.493546
1885455,ENSMUST00000239525.1,9041,8910,GUG,V,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.497849


# Odd ratios & FDR

In [114]:
avg_count_table = slct_codon_table[['transcript_id','reltostart-asite','codon_num']].copy()
for day in ['D0','D4','D8']:
    avg_count_table[f'{day}_n'] = slct_codon_table[[f'{day}{rep}_n' for rep in ['a','b','c']]].mean(axis=1)
    avg_count_table[f'{day}_N'] = slct_codon_table[[f'{day}{rep}_N' for rep in ['a','b','c']]].mean(axis=1)

display(avg_count_table)

odd_ratio = avg_count_table[['transcript_id','reltostart-asite']].copy()
for day in ['D4','D8']:
    odd_ratio[day+'_OR'] = (avg_count_table[f'{day}_n'] / avg_count_table[f'D0_n']) / ((avg_count_table[f'{day}_N'] - avg_count_table[f'{day}_n']) / (avg_count_table[f'D0_N'] - avg_count_table[f'D0_n']))

display(odd_ratio)

,transcript_id,reltostart-asite,codon_num,D0_n,D0_N,D4_n,D4_N,D8_n,D8_N
0,ENSMUST00000000001.5,33,334,2.666667,1343.333333,1.000000,1450.000000,1.666667,1155.666667
1,ENSMUST00000000001.5,36,334,0.666667,1343.333333,1.333333,1450.000000,1.333333,1155.666667
2,ENSMUST00000000001.5,39,334,2.000000,1343.333333,0.333333,1450.000000,3.333333,1155.666667
3,ENSMUST00000000001.5,42,334,0.000000,1343.333333,2.000000,1450.000000,0.000000,1155.666667
4,ENSMUST00000000001.5,45,334,1.666667,1343.333333,1.333333,1450.000000,1.000000,1155.666667
...,...,...,...,...,...,...,...,...,...
3476,ENSMUST00000239525.1,10461,3481,0.666667,10038.000000,0.000000,6427.333333,0.000000,1866.666667
3477,ENSMUST00000239525.1,10464,3481,1.000000,10038.000000,1.333333,6427.333333,0.000000,1866.666667
3478,ENSMUST00000239525.1,10467,3481,6.000000,10038.000000,1.666667,6427.333333,0.000000,1866.666667
3479,ENSMUST00000239525.1,10470,3481,7.000000,10038.000000,2.666667,6427.333333,1.333333,1866.666667


,transcript_id,reltostart-asite,D4_OR,D8_OR
0,ENSMUST00000000001.5,33,0.346963,0.726098
1,ENSMUST00000000001.5,36,1.853659,2.326307
2,ENSMUST00000000001.5,39,0.154212,1.940025
3,ENSMUST00000000001.5,42,inf,NaN
4,ENSMUST00000000001.5,45,0.740911,0.697171
...,...,...,...,...
3476,ENSMUST00000239525.1,10461,0.000000,0.000000
3477,ENSMUST00000239525.1,10464,2.082581,0.000000
3478,ENSMUST00000239525.1,10467,0.433677,0.000000
3479,ENSMUST00000239525.1,10470,0.594791,1.024303


In [115]:
from scipy.stats import fisher_exact

avg_count_table_round = avg_count_table.copy()
avg_count_table_round.iloc[:,3:] = avg_count_table_round.iloc[:,3:].round().astype(int)

D4_OR = []
D4_p = []
D8_OR = []
D8_p = []
for i in range(len(avg_count_table_round)):
    row = avg_count_table_round.iloc[i,:]

    D4_FE = fisher_exact( [[row['D4_n'],row['D0_n']], [row['D4_N']-row['D4_n'],row['D0_N']-row['D0_n']]] )
    D8_FE = fisher_exact( [[row['D8_n'],row['D0_n']], [row['D8_N']-row['D8_n'],row['D0_N']-row['D0_n']]] )

    D4_OR.append(D4_FE[0])
    D4_p.append(D4_FE[1])
    D8_OR.append(D8_FE[0])
    D8_p.append(D8_FE[1])

FE_results = avg_count_table_round[['transcript_id','reltostart-asite']].copy()

FE_results['D4_OR'] = D4_OR
FE_results['D4_p'] = D4_p
FE_results['D8_OR'] = D8_OR
FE_results['D8_p'] = D8_p

print('Fisher test done')

# FDR calculation for each transcript
FE_results_FDR = pd.DataFrame(columns=['transcript_id','reltostart-asite','D4_OR','D4_p','D4_FDR','D8_OR','D8_p','D8_FDR'])
for T_ID in set(FE_results['transcript_id']):
    chunk = FE_results[ FE_results['transcript_id']==T_ID ]
    
    # p-value of day 4
    chunk = chunk.sort_values('D4_p')
    chunk['p-rank'] = [*range(1,len(chunk)+1)]
    chunk = chunk.iloc[::-1]

    adj_p = []
    data_len = len(chunk)
    tmp = chunk[['reltostart-asite','D4_p','p-rank']].set_index('reltostart-asite').T
    prev = 1
    for ID in tmp:
        now = tmp[ID][0] * data_len / tmp[ID][1]
        adj_p.append( min(prev, now) )
        prev = min(prev, now)
    chunk['D4_FDR'] = adj_p
    chunk = chunk.drop(columns='p-rank')

    # p-value of day 8
    chunk = chunk.sort_values('D8_p')
    chunk['p-rank'] = [*range(1,len(chunk)+1)]
    chunk = chunk.iloc[::-1]

    adj_p = []
    data_len = len(chunk)
    tmp = chunk[['reltostart-asite','D8_p','p-rank']].set_index('reltostart-asite').T
    prev = 1
    for ID in tmp:
        now = tmp[ID][0] * data_len / tmp[ID][1]
        adj_p.append( min(prev, now) )
        prev = min(prev, now)
    chunk['D8_FDR'] = adj_p
    chunk = chunk.drop(columns='p-rank')

    FE_results_FDR = pd.concat([FE_results_FDR, chunk])

FE_results_FDR = FE_results_FDR.sort_values(by=['transcript_id','reltostart-asite']).reset_index(drop=True)

FE_results_FDR.to_csv('adi_stalling-change_FEresult.tsv', sep='\t', index=False)
display(FE_results_FDR)

Fisher test done


/home/daehwa/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/home/daehwa/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/home/daehwa/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/home/daehwa/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes

,transcript_id,reltostart-asite,D4_OR,D4_p,D4_FDR,D8_OR,D8_p,D8_FDR
0,ENSMUST00000000001.5,33,0.308259,0.356685,1.0,0.774119,1.0,1.0
1,ENSMUST00000000001.5,36,0.926156,1.0,1.0,1.161905,1.0,1.0
2,ENSMUST00000000001.5,39,0.0,0.231122,1.0,1.744579,0.667567,1.0
3,ENSMUST00000000001.5,42,inf,0.500555,1.0,NaN,1.0,1.0
4,ENSMUST00000000001.5,45,0.462733,0.611056,1.0,0.580519,1.0,1.0
...,...,...,...,...,...,...,...,...
2238001,ENSMUST00000239525.1,10461,0.0,1.0,1,0.0,1.0,1.0
2238002,ENSMUST00000239525.1,10464,1.561936,1.0,1,0.0,1.0,1.0
2238003,ENSMUST00000239525.1,10467,0.520467,0.494876,1,0.0,0.598884,1.0
2238004,ENSMUST00000239525.1,10470,0.669209,0.74959,1,0.767953,1.0,1.0


In [4]:
codon_table = pd.DataFrame(columns=['transcript_id','asite','reltostart-asite','codon-asite','aa-asite'])
for SP in [day+rep for day in ['0','4','8'] for rep in ['a','b','c']]:
    codon_data = pd.read_csv(f'/Data_2/Jun/Adipocytes/rpf/novaseq/D{SP}.rep.codons.data.txt', sep='\t')
    codon_data = codon_data[['transcript_id','asite','reltostart-asite','codon-asite','aa-asite']]
    codon_table = pd.merge(codon_table, codon_data, on=['transcript_id','asite','reltostart-asite','codon-asite','aa-asite'], how='outer')

FE_results_FDR_addinfo = pd.merge(codon_table, FE_results_FDR, on=['transcript_id','reltostart-asite'])

FE_results_FDR_addinfo.to_csv('adi_stalling-change_FEresult_addinfo.tsv', sep='\t', index=False)
display(FE_results_FDR_addinfo)

,transcript_id,reltostart-asite,D4_OR,D4_p,D4_FDR,D8_OR,D8_p,D8_FDR
0,ENSMUST00000000001.5,33,0.308259,0.356685,1.0,0.774119,1.000000,1.0
1,ENSMUST00000000001.5,36,0.926156,1.000000,1.0,1.161905,1.000000,1.0
2,ENSMUST00000000001.5,39,0.000000,0.231122,1.0,1.744579,0.667567,1.0
3,ENSMUST00000000001.5,42,inf,0.500555,1.0,NaN,1.000000,1.0
4,ENSMUST00000000001.5,45,0.462733,0.611056,1.0,0.580519,1.000000,1.0
...,...,...,...,...,...,...,...,...
2238001,ENSMUST00000239525.1,10461,0.000000,1.000000,1.0,0.000000,1.000000,1.0
2238002,ENSMUST00000239525.1,10464,1.561936,1.000000,1.0,0.000000,1.000000,1.0
2238003,ENSMUST00000239525.1,10467,0.520467,0.494876,1.0,0.000000,0.598884,1.0
2238004,ENSMUST00000239525.1,10470,0.669209,0.749590,1.0,0.767953,1.000000,1.0


,transcript_id,asite,reltostart-asite,codon-asite,aa-asite,D4_OR,D4_p,D4_FDR,D8_OR,D8_p,D8_FDR
0,ENSMUST00000000001.5,174,33,GCG,A,0.308259,0.356685,1.0,0.774119,1.000000,1.0
1,ENSMUST00000000001.5,177,36,GUG,V,0.926156,1.000000,1.0,1.161905,1.000000,1.0
2,ENSMUST00000000001.5,180,39,GAG,E,0.000000,0.231122,1.0,1.744579,0.667567,1.0
3,ENSMUST00000000001.5,186,45,AGC,S,0.462733,0.611056,1.0,0.580519,1.000000,1.0
4,ENSMUST00000000001.5,189,48,AAG,K,0.000000,0.480845,1.0,2.325823,0.599013,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1885452,ENSMUST00000239525.1,4271,4140,UCU,S,NaN,1.000000,1.0,NaN,1.000000,1.0
1885453,ENSMUST00000239525.1,5324,5193,GCG,A,NaN,1.000000,1.0,NaN,1.000000,1.0
1885454,ENSMUST00000239525.1,8915,8784,CGC,R,NaN,1.000000,1.0,inf,0.156825,1.0
1885455,ENSMUST00000239525.1,9041,8910,GUG,V,NaN,1.000000,1.0,NaN,1.000000,1.0


In [116]:
# FE_results_FDR = pd.read_csv('/Data_2/Daehwa/Adipocyte/Analysis/Ribosome_stalling/v20230730/adi_stalling-change_FEresult.tsv', sep='\t')
# display(FE_results_FDR)

FE_results_FDR_compact = FE_results_FDR[ (FE_results_FDR['D4_FDR']<0.1) | (FE_results_FDR['D8_FDR']<0.1) ].copy()
# display(FE_results_FDR_compact)

codon_table = pd.DataFrame(columns=['transcript_id','asite','reltostart-asite','codon-asite','aa-asite'])
for SP in [day+rep for day in ['0','4','8'] for rep in ['a','b','c']]:
    codon_data = pd.read_csv(f'/Data_2/Jun/Adipocytes/rpf/novaseq/D{SP}.rep.codons.data.txt', sep='\t')
    codon_data = codon_data[['transcript_id','asite','reltostart-asite','codon-asite','aa-asite']]
    codon_table = pd.merge(codon_table, codon_data, on=['transcript_id','asite','reltostart-asite','codon-asite','aa-asite'], how='outer')

FE_results_FDR_compact = pd.merge(codon_table, FE_results_FDR_compact, on=['transcript_id','reltostart-asite'])

FE_results_FDR_compact.to_csv('adi_stalling-change_FEresult_sig.tsv', sep='\t', index=False)
display(FE_results_FDR_compact)

,transcript_id,asite,reltostart-asite,codon-asite,aa-asite,D4_OR,D4_p,D4_FDR,D8_OR,D8_p,D8_FDR
0,ENSMUST00000000001.5,267,126,GAA,E,2.663253,0.002675,0.369862,4.07855,0.000004,0.000737
1,ENSMUST00000000001.5,330,189,GAC,D,0.232194,0.000008,0.002639,0.409897,0.003295,0.198856
2,ENSMUST00000000001.5,429,288,GAA,E,1.507714,0.005393,0.450347,2.074848,0.0,0.000109
3,ENSMUST00000000001.5,1029,888,GAA,E,1.995022,0.138542,1.0,4.391403,0.000157,0.017433
4,ENSMUST00000000080.8,310,153,GAA,E,1.550548,0.402057,1.0,3.975017,0.00009,0.013367
...,...,...,...,...,...,...,...,...,...,...,...
32771,ENSMUST00000119704.2,187,126,GAA,E,inf,0.011079,0.119103,inf,0.010764,0.077145
32772,ENSMUST00000179865.8,661,477,GAA,E,inf,0.034824,0.236098,inf,0.002695,0.024047
32773,ENSMUST00000225805.2,1227,1065,GAA,E,inf,0.002096,1,inf,0.000143,0.073177
32774,ENSMUST00000239402.2,188,165,UAU,Y,inf,0.063508,0.457,inf,0.008853,0.06841
